In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Sequential 
from tensorflow.keras.models import Model
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import load_model

In [5]:
#Input Size of image 
Image_size = [224,224]

#training and valid path 
train_path = '/content/drive/MyDrive/Dog_cat with Transfer learning/train'
valid_path = '/content/drive/MyDrive/Dog_cat with Transfer learning/test' 

In [6]:
Image_size + [3]

[224, 224, 3]

In [7]:
#pulling out trained weights from VGG16
#include top = False as we will not use FC and O/p of VGG16 we will use till last max pooling 
#weights = imagenet name to get trained weights
#input_shape = Shape input for model
vgg16 = VGG16(include_top = False, weights='imagenet', input_shape= Image_size + [3])

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
# to stop training of all the weight in VGG16 layers
for layer in vgg16.layers:
  layer.trainable = False


In [9]:
#to get the number of output classes
folders = glob('/content/drive/MyDrive/Dog_cat with Transfer learning/train/*')

In [ ]:
folders

['/content/drive/MyDrive/Dog_cat with Transfer learning/train/Cats',
 '/content/drive/MyDrive/Dog_cat with Transfer learning/train/Dogs']

In [ ]:
len(folders)

2

In [11]:
#Adding our flatten layer after max pooling layer of VGG16
X = Flatten()(vgg16.output)

In [12]:
#creating output layer
prediction = Dense(len(folders),activation = "softmax")(X)


In [13]:
#creating model object
model2 = Model(inputs = vgg16.input,outputs = prediction)

In [14]:
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
#FITTING THE MODEL
model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Dog_cat with Transfer learning/train',
                                                 target_size=(264,264),batch_size=32,class_mode="categorical")

Found 25000 images belonging to 2 classes.


In [18]:
#Scaling test data before getting prediction from trained model
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Dog_cat with Transfer learning/test',target_size=(264,264),batch_size = 32,class_mode="categorical")

Found 25000 images belonging to 2 classes.


In [ ]:
len(test_set)

782

In [19]:
#Fit the model
#Always use model.fit_generator while using ImageDataGenerator
#steps_per_epocs - In every epocs it will execute n  times here n is "steps_per_epochs"
r = model2.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)

)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50


InvalidArgumentError: ignored

In [ ]:
#plotting training and validation loss 
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label = 'train_loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
ply.legend()
plt.show()
plt.savefig('LossVal_loss')


In [ ]:
#plotting training and validation accuracy
import matplotlib.pyplot as plt
plt.plot(r.history['accuracy'], label = 'train_accuracy')
plt.plot(r.history['val_accuracy'], label = 'val_accuracy')
ply.legend()
plt.show()
plt.savefig('train_valid_accuracy')


In [ ]:

# save it as a h5 file


from tensorflow.keras.models import load_model

cnn.save('model_rcat_transfer.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/test/Cat/kitten-510651.jpg', target_size = (264,264))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [ ]:
#as we have sue softmax activation function so it will give probablity of every category so there wiil bee two 
#output like[0.9953,0.0047]
a = np.argmax(result)

In [ ]:
#so a will contain values on two indices a[0],a[1]
if a == [0]:
    print("The image classified is dog")
else:
    print("The image classified is cat")